
DONE: copy code from https://github.com/briancullinan2/quake3-discord-bot/tree/main/discordApi

DISCORD: https://discord.gg/taHjA7Cg

how to authorize and connect to discord?

authorize discord?

discord authorize?



In [ ]:
const {request} = require('gaxios')
const WebSocket = require('ws')
const {delay, wait, timeout} = importer.import('discord utilities')
const {
  gatewayIdentified, gatewayClose, gatewayMessage,
} = importer.import('discord gateway')
const {
  TOKEN, DEFAULT_API, DEFAULT_RATE
} = importer.import('discord configuration')

var ws = false
var wsConnecting = false
var previousRequest = 0

async function requestAuthQ(outgoing) {
  await authorizeGateway()
  if(typeof outgoing.headers == 'undefined')
    outgoing.headers = {}
  outgoing.headers['Authorization'] = `Bot ${TOKEN}`
  outgoing.url = DEFAULT_API + outgoing.url
  previousRequest = await delay(previousRequest, DEFAULT_RATE)
  var resolveRequest
  resolveRequest = async () => {
    var result
    try {
      //console.log('Discord request', outgoing)
      result = (await request(outgoing)).data
    } catch (e) {
      // check result for rate limit and re-run this request in a queue
      if(e.code == 429) {
        console.log('Delayed request', e.response.data.retry_after)
        await timeout(e.response.data.retry_after * 1000)
        return await resolveRequest()
      } else {
        console.log(e)
        if(e.response) {
          console.log(e.response.data.errors)
        }
        throw e
      }
    }
    return result
  }
  return await resolveRequest()
}

async function gatewayUrl() {
  // TODO: return the same result if queried less than 1 second ago
  // doesn't use requestAuthQ because that would create an infinite loop
  var result = await request({
    headers: {
      'Authorization': `Bot ${TOKEN}`
    },
    method: 'GET',
    url: `${DEFAULT_API}gateway/bot`
  })
  return result.data
}

function gatewayOpen() {
  console.log('Connecting to Discord')
}

async function authorizeGateway() {
  if(wsConnecting) {
    var result = await wait(() => ws && ws.identified, 3000)
    if(!result)
      return await authorizeGateway()
    else
      return ws
  } else if (ws && ws.readyState == 1 && ws.identified) {
    return ws
  }

  wsConnecting = true
  try {
    ws = new WebSocket((await gatewayUrl()).url)
    ws.identified = false
  } catch (e) {
    console.log('Authorize error', e.message)
    ws = false
    wsConnecting = false
    return
  }
  ws.on('open', gatewayOpen)
  ws.on('message', gatewayMessage.bind(null, ws, authorizeGateway))
  ws.on('close', gatewayClose.bind(null, ws, authorizeGateway))
  await wait(() => ws.identified, 3000)
  wsConnecting = false
  return ws
}

function closeGateway() {
  gatewayClose(ws)
}

module.exports = {
  authorizeGateway,
  request: requestAuthQ,
  gatewayUrl,
  closeGateway,
}






discord bot configuration?


In [ ]:
var fs = require('fs')
var path = require('path')

var DEFAULT_GUILD = process.env.DEFAULT_GUILD || '319817668117135362'
var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || '1328142980967563337' // 366715821654933515
var DEFAULT_APPLICATION = process.env.DEFAULT_APPLICATION || '1328141471840206899'
var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v10/'
var MESSAGE_TIME = process.env.DEFAULT_TIME || 1000 * 60 * 2 // * 60 // 2 minutes to respond
var DEFAULT_RATE = 1000 / 50 // from discord documentation
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var DEFAULT_USERNAME = 'Orbb'
var MESSAGES_START = 1420070400000

var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
  tokenPath = path.resolve('./discord-bot.txt')
} else {
  tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}

var secretPath
if(fs.existsSync('./discord.txt')) {
  secretPath = path.resolve('./discord.txt')
} else {
  secretPath = path.join(PROFILE_PATH, '.credentials/discord.txt')
}


var TOKEN = fs.readFileSync(tokenPath).toString('utf-8').trim()
var SECRET = fs.readFileSync(secretPath).toString('utf-8').trim()

module.exports = {
  DEFAULT_GUILD,
  DEFAULT_CHANNEL,
  DEFAULT_APPLICATION,
  DEFAULT_API,
  MESSAGE_TIME,
  DEFAULT_RATE,
  PROFILE_PATH,
  TOKEN,
  SECRET,
  DEFAULT_USERNAME,
  MESSAGES_START
}



discord api?


In [ ]:
var {DEFAULT_CHANNEL, DEFAULT_USERNAME} = importer.import('discord configuration')
var {
  authorizeGateway, authorizeUrl, closeGateway, request
} = importer.import('discord authorize')

async function triggerTyping(channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'POST',
    url: `channels/${channelId}/typing`
  })
}

module.exports = {
  DEFAULT_USERNAME,
  authorizeGateway,
  authorizeUrl,
  closeGateway,
  triggerTyping,
  ... {
    userChannels, guildChannels, channelMessages, deleteChannel
  } = importer.import('discord channels'),
  ... {
    userGuilds, userConnections, getGuildRoles
  } = importer.import('discord guilds'),
  ... {
    createMessage, updateMessage,
    getPins, pinMessage, unpinMessage
  } = importer.import('discord messages'),
  ... {
    registerCommand, getCommands, deleteCommand,
    interactionResponse, updateInteraction, updateCommand,
  } = importer.import('discord commands'),
  ... {
    createThread, archivedThreads, activeThreads,
    addThreadMember,
  } = importer.import('discord threads'),
  ... {
    getUser
  } = importer.import('discord users'),
}





discord gateway?


In [ ]:
const {TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const systemUsage = importer.import('system usage report')

var indentifyTimer
var privateChannels = {}
var interactions = {}
const interactionsCommands = {}
var cancelConnection // if gateway doesn't respond properly, close web socket
var heartbeat
var resources
var seq = 0

function sendHeartbeat(ws, reconnect) {
  if(!ws) return
  console.log('Sending heartbeat')
  ws.send(JSON.stringify({
    op: 1,
    d: seq
  }))
  cancelConnection = setTimeout(gatewayClose.bind(null, ws, reconnect), 4000)
}

function gatewayMessage(ws, reconnectGateway, message) {
  var msgBuff = new Buffer.from(message)
  var gateway = JSON.parse(msgBuff.toString('utf-8'))
  if(gateway.s) seq = gateway.s
  if(gateway.d && gateway.d.seq) seq = gateway.d.seq
  
  console.log('Gateway message', gateway)
  
  if(gateway.op == 10) {
    ws.identified = true
    heartbeat = setInterval(sendHeartbeat.bind(null, ws, reconnectGateway), gateway.d.heartbeat_interval)
    resources = setInterval(systemUsage, 1000)
    ws.send(JSON.stringify({
      op: 2,
      intents: [
        'DIRECT_MESSAGES', 'DIRECT_MESSAGE_REACTIONS', 
        'GUILD_MESSAGES', 'GUILD_MESSAGE_REACTIONS', 
        'GUILDS', 'THREAD_UPDATE', 'THREAD_CREATE',
        'THREAD_DELETE', 'THREAD_LIST_SYNC', 'THREAD_MEMBER_UPDATE',
        'THREAD_MEMBERS_UPDATE', 'MESSAGE_CREATE', 'MESSAGE_UPDATE',
        'GUILD_PRESENCES',
      ],
      d: {
        token: TOKEN,
        properties: {
          "$os": "linux",
          "$browser": "nodejs",
          "$device": "quake3"
        }
      }
    }))
    return
  } else if (gateway.op === 7) { // should reconnect
    console.log('Reconnecting...')
    gatewayClose(ws, reconnectGateway)
    return
  } else if (gateway.op === 0 || gateway.op === 9) {
    if(gateway.t == 'MESSAGE_CREATE') {
      // guild ID can only be null if it is a personal message
      if(typeof gateway.d.guild_id == 'undefined') {
        privateChannels[gateway.d.channel_id] = Date.now()
        if(gateway.d.author.id != DEFAULT_APPLICATION) {
          interactionsCommands['promptPrivate'](gateway.d)
        }
      } else if(gateway.d.content.includes('@' + DEFAULT_APPLICATION)
        && typeof interactionsCommands['promptMention'] != 'undefined'
      ) {
        interactionsCommands['promptMention'](gateway.d)
      }
    }
    if(gateway.t == 'INTERACTION_CREATE') {
      if(typeof interactions[gateway.d.channel_id] == 'undefined')
        interactions[gateway.d.channel_id] = []
      interactions[gateway.d.channel_id].push(gateway.d)
      if(typeof interactionsCommands[gateway.d.data.name] != 'undefined') {
        interactionsCommands[gateway.d.data.name](gateway.d)
      }
      interactionResponse(gateway.d.id, gateway.d.token)
    }
    if(gateway.t == 'EMBEDDED_ACTIVITY_UPDATE_V2'
      && typeof interactionsCommands['startActivity'] != 'undefined'
      && typeof interactionsCommands['endActivity'] != 'undefined'
    ) {
      if(gateway.d.participants.length) {
        Promise.resolve(interactionsCommands['startActivity'](gateway.d))
      } else {
        Promise.resolve(interactionsCommands['endActivity'](gateway.d))
      }
    }
    return
  } else if (gateway.op === 11) {
    clearTimeout(cancelConnection)
    return
  }
  console.log('Unrecognized gateway', gateway)
}

function gatewayClose(ws, reconnect) {
  console.log('Discord disconnected')
  if(indentifyTimer)
    clearInterval(indentifyTimer)
  if(heartbeat)
    clearInterval(heartbeat)
  if(resources)
    clearInterval(resources)
  if(ws.readyState == 1)
    ws.close()
  if(reconnect) {
    setTimeout(reconnect, 1000)
  }
  if(typeof interactionsCommands['endActivity'] != 'undefined') {
    Promise.resolve(interactionsCommands['endActivity']())
  }
  ws.identified = false
  return
}

module.exports = {
  gatewayClose,
  gatewayMessage,
  privateChannels,
  interactions,
  interactionsCommands,
}




discord messages?


In [ ]:

var {DEFAULT_CHANNEL} = importer.import('discord configuration')
var {request} = importer.import('discord authorize')

async function createMessage(message, channelId = DEFAULT_CHANNEL) {
  var params = typeof message == 'string' ? ({
    'content': message
  }) : message
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `channels/${channelId}/messages`,
    data: JSON.stringify(params)
  })
}

async function deleteMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'DELETE',
    url: `channels/${channelId}/messages/${messageId}`
  })
}

async function updateMessage(message, messageId, channelId = DEFAULT_CHANNEL) {
  var params = typeof message == 'string' ? ({
    'content': message
  }) : message
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'PATCH',
    url: `channels/${channelId}/messages/${messageId}`,
    data: JSON.stringify(params)
  })
}

async function getPins(channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'GET',
    url: `channels/${channelId}/pins`
  })
}

async function pinMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'PUT',
    url: `channels/${channelId}/pins/${messageId}`
  })
}

async function unpinMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'DELETE',
    url: `channels/${channelId}/pins/${messageId}`
  })
}

module.exports = {
  createMessage,
  deleteMessage,
  updateMessage,
  getPins,
  pinMessage,
  unpinMessage
}





discord channels?



In [ ]:
var {
  DEFAULT_GUILD, DEFAULT_CHANNEL, MESSAGE_TIME,
  MESSAGES_START, 
} = importer.import('discord configuration')
var {request} = importer.import('discord authorize')

async function userChannels(userId = '@me') {
  return await request({
    method: 'GET',
    url: `channels/${userId}`
  })
}

async function guildChannels(guildId = DEFAULT_GUILD) {
  return await request({
    method: 'GET',
    url: `guilds/${guildId}/channels`
  })
}

async function channelMessagesB(channelId = DEFAULT_CHANNEL, messageTime = MESSAGE_TIME) {
  var params = {
    limit: 100,
    after: messageTime.toString()
  };
  var messages = await request({
    method: 'GET',
    url: `channels/${channelId}/messages`,
    params
  })
  if(messages.length == 100) {
    messages = messages.concat(await channelMessagesB(channelId, BigInt(messages[0].id) + BigInt(1)))
  }
  return messages
}

async function channelMessages(channelId = DEFAULT_CHANNEL, messageTime = MESSAGE_TIME) {
  var params = {
    limit: 100,
    after: (BigInt(Date.now() - MESSAGES_START - messageTime) << BigInt(22)).toString()
  };
  var messages = await request({
    method: 'GET',
    url: `channels/${channelId}/messages`,
    params
  })
  if(messages.length == 100) {
    messages = messages.concat(await channelMessagesB(channelId, BigInt(messages[0].id) + BigInt(1)))
  }
  return messages
}

async function deleteChannel(channelId) {
  return await request({
    method: 'DELETE',
    url: `channels/${channelId}`
  })
}

module.exports = {
  userChannels,
  guildChannels,
  channelMessages,
  deleteChannel
}




discord commands?


In [ ]:

const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const {request} = importer.import('discord authorize')
const {timeout} = importer.import('discord utilities')


async function registerCommand(cmd, desc, guildId = null) {
  // TODO: guild specific commands
  //url = "https://discord.com/api/v8/applications/<my_application_id>/guilds/<guild_id>/commands"
  var json
  if(typeof cmd == 'object') {
    json = cmd
  } else {
    json = {
      'name': cmd,
      'description': desc
    }
  }
  console.log('Registering command ', json.name)
  await timeout(2000)
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands`
      : `applications/${DEFAULT_APPLICATION}/commands`,
    data: JSON.stringify(json)
  })
}

async function interactionResponse(interactionId, interactionToken) {
  var json = {
    'type': 5
  }
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `interactions/${interactionId}/${interactionToken}/callback`,
    data: JSON.stringify(json)
  })
}

async function getCommands(guildId = null) {
  return await request({
    method: 'GET',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands`
      : `applications/${DEFAULT_APPLICATION}/commands`
  })
}

async function getCommand(commandId, guildId = null) {
  return await request({
    method: 'GET',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`
  })
}

async function updateInteraction(message, interactionId, interactionToken) {
  var json = typeof message == 'string' ? ({
      'content': message
    }) : message
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'PATCH',
    url: `webhooks/${DEFAULT_APPLICATION}/${interactionToken}/messages/@original`,
    data: JSON.stringify(json)
  })
}

async function updateCommand(cmd, desc, commandId, guildId = null) {
  if(typeof cmd == 'object') {
    json = cmd
  } else {
    json = {
      'name': cmd,
      'description': desc
    }
  }
  console.log('Updating command ', json.name)
  await timeout(2000)
  return await request({
    method: 'PATCH',
    headers: {
      'Content-Type': 'application/json'
    },
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`,
    data: JSON.stringify(json)
  })
}

async function deleteCommand(commandId, guildId = null) {
  console.log('Deleting command ', commandId)
  return await request({
    method: 'DELETE',
    url: guildId 
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`
  })
}

module.exports = {
  registerCommand,
  interactionResponse,
  getCommands,
  getCommand,
  updateInteraction,
  deleteCommand,
  updateCommand,
}



discord guilds?


In [ ]:
var {DEFAULT_GUILD} = importer.import('discord configuration')
var {request} = importer.import('discord authorization')


async function userGuilds(userId = '@me') {
  return await request({
    method: 'GET',
    url: `users/${userId}/guilds`
  })
}

async function getGuildRoles(guildId = DEFAULT_GUILD) {
  return await request({
    method: 'GET',
    url: `guilds/${guildId}/roles`
  })
}

async function userConnections(userId = '@me') {
  return await request({
    method: 'GET',
    url: `users/${userId}/connections`
  })
}

module.exports = {
  userGuilds,
  getGuildRoles,
  userConnections
}



discord threads?


In [ ]:
var {DEFAULT_CHANNEL} = importer.import('discord configuration')
var {request} = importer.import('discord authorize')

async function createThread(name, channelId = DEFAULT_CHANNEL) {
  var json = {
    'name': name,
    'type': 11,
    'auto_archive_duration': 60
  }
  return await request({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `channels/${channelId}/threads`,
    data: JSON.stringify(json)
  })
}

async function archivedThreads(channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'GET',
    url: `channels/${channelId}/threads/archived/public`
  })
}

async function activeThreads(channelId = DEFAULT_CHANNEL) {
  return await request({
    method: 'GET',
    url: `channels/${channelId}/threads/active`
  })
}

async function addThreadMember(memberId, channelId) {
  return await request({
    method: 'PUT',
    url: `/channels/${channelId}/thread-members/${memberId}`
  })
}

module.exports = {
  createThread,
  archivedThreads,
  activeThreads,
  addThreadMember,
}



discord users?


In [ ]:
var {request} = importer.import('discord authorize')

async function getUser(userId = '@me') {
  return await request({
    method: 'GET',
    url: `/users/${userId}`
  })
}

module.exports = {
  getUser,
}



discord utilities?


In [ ]:
var timers = {}
var mainTimer = setInterval(callResolve, 20)

function callResolve() {
  var now = Date.now()
  var times = Object.keys(timers)
  for(var i = 0; i < times.length; i++) {
    if(now > times[i]) {
      try {
        Promise.resolve(timers[times[i]]())
      } catch (e) {
        console.log('timer failed', e)
        throw e
      }
      delete timers[times[i]]
      return
    }
  }
}

function addResolve(resolve, time) {
  while(typeof timers[time] != 'undefined') {
    time++
  }
  timers[time] = resolve
}

async function timeout(delay) {
  var now = Date.now()
  await new Promise(resolve => addResolve(resolve, now + delay))  
}

async function delay(prev, delay) {
  var now = Date.now()
  if(now - prev < delay)
    await new Promise(resolve => addResolve(resolve, now + (delay - (now - prev))))
  return Date.now()
}

async function wait(until, delay) {
  var waitTimer
  var waitCount = 0
  var result
  var now = Date.now()
  var delayed = now + delay
  while(!result && now < delayed) {
    await timeout(100)
    result = await until()
    now = Date.now()
  }
  return result
}

module.exports = {
  timeout,
  delay,
  wait,
}



add some basic commands

sync discord commands?



In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {
  doInteraction, doHallucinate, doImagineProxy, doMention, doPrivate
} = importer.import('discord stability proxy')
const {doDiscordPdf} = importer.import('discord pdf converter')
const {
  doInterpret, doSystemUsage, doDistill, doInstruct, 
  doDeceive, doReason, doArgument, doMesh, doVoice
} = importer.import('discord notebook connector')
const {discordExpress, closeExpress, getInstance} = importer.import('discord express server')

const ALL_COMMANDS = [
  'hello',
  'prompt',
  'imagine',
  'hallucinate',
  'pdf',
  'interpret',
  'usage',
  'distill',
  'instruct',
  'deceive',
  'reason',
  'argue',
  'mesh',
  'voice',
  'launch',

]

// bot commands using new API, same names as above but lower-case
async function syncCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress


  if(!commands.includes('hello')) {
    await registerCommand('hello', 'Check if Orbb is awake.', guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'hello')[0]
    //await updateCommand(cmd.name, cmd.description, cmd.id, guildId)
  }
  interactionsCommands['hello'] = doInteraction

  cmdDef = {
    'name': 'prompt',
    'description': 'Prompt an llm.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the llm with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('prompt')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'prompt')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description 
      || cmdDef.options.length != cmd.options.length
      || JSON.stringify(cmdDef.options) != JSON.stringify(cmd.options.map(o => ({
        name: o.name,
        description: o.description,
        required: o.required,
        type: o.type
      })))) {
      debugger
      await updateCommand(cmdDef, null, cmd.id, guildId)
    }
  }
  interactionsCommands['prompt'] = doInteraction
  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate

  cmdDef = {
    'name': 'imagine',
    'description': 'Prompt an image generator.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt stability diffusion with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('imagine')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'imagine')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['imagine'] = doImagineProxy

  cmdDef = {
    'name': 'hallucinate',
    'description': 'Prompt an image generator with weirder content.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the llm with then use with stable diffusion.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('hallucinate')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'hallucinate')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['hallucinate'] = doHallucinate

  cmdDef = {
    'name': 'pdf',
    'description': 'Convert a website to PDF.',
    'options': [
      {
        'name': 'address',
        'description': 'Http address to convert.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('pdf')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'pdf')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['pdf'] = doDiscordPdf


  cmdDef = {
    'name': 'interpret',
    'description': 'Interpret what a tool will do.',
    'options': [
      {
        'name': 'search',
        'description': 'Search function for the tool name.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('interpret')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'interpret')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['interpret'] = doInterpret


  cmdDef = {
    'name': 'usage',
    'description': 'View system usage.',
    'options': []
  }
  if(!commands.includes('usage')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'usage')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['usage'] = doSystemUsage


  cmdDef = {
    'name': 'distill',
    'description': 'Distill with DeepSeek R1 8B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the DeepSeek R1 8B with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('distill')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'distill')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['distill'] = doDistill


  cmdDef = {
    'name': 'instruct',
    'description': 'Meta Instruct 70B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the Meta Instruct 70B with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('instruct')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'instruct')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['instruct'] = doInstruct

  cmdDef = {
    'name': 'deceive',
    'description': 'DeepSeek 7B Chat.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt deceptive LLM with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('deceive')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'deceive')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['deceive'] = doDeceive

  cmdDef = {
    'name': 'reason',
    'description': 'Qwen 2.5 14B M1.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt Qwen reasoning LLM.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('reason')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'reason')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['reason'] = doReason



  cmdDef = {
    'name': 'argue',
    'description': 'Start an argument between deceptive DeepSeek 7B Chat and Meta Llama 8B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt argumentative LLM with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('argue')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'argue')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['argue'] = doArgument


  cmdDef = {
    'name': 'mesh',
    'description': 'LLaMa-Mesh gguf.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt mesh generator with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('mesh')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'mesh')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['mesh'] = doMesh


  cmdDef = {
    'name': 'voice',
    'description': 'OuteTTS 0.3 1B/Llasa 3B gguf.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to convert to audio.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('voice')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'voice')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['voice'] = doVoice


  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

async function deleteCommands(guildId) {
  var toRemove = await getCommands(guildId)
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }
}


module.exports = {
  syncCommands,
  deleteCommands,
  
}


this is getting long.

update discord interaction with attachments?


In [ ]:
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const {request: requestDiscord} = importer.import('discord authorize')
const { safeurl } = importer.import('domain cache tools')
const mime = require('mime-types');


async function patchInteractionImageAttachment(prompt, buff, token, content = 'image/png') {
  return await requestDiscord({
    headers: {
      'Content-Type': 'multipart/form-data; boundary=boundary123'
    },
    method: 'PATCH',
    url: `webhooks/${DEFAULT_APPLICATION}/${token}/messages/@original`,
    body: `--boundary123
Content-Disposition: form-data; name="payload_json"
Content-Type: application/json

${JSON.stringify({
  content: 'Your ' + (content.includes('image') ? 'image' : content) + ' delivered: *' + prompt + '*',
  attachments: [{
    "id": 0,
    "description": prompt,
    "filename": safeurl(prompt) + '.' + mime.extension(content)
  }],
})}
--boundary123
Content-Disposition: form-data; name="files[0]"; filename="${safeurl(prompt)}.${mime.extension(content)}"
Content-Transfer-Encoding: base64
Content-Type: ${content}

${buff.toString('base64')}
--boundary123--`
  })
}


module.exports = {
  patchInteractionImageAttachment
}



some sample interactions.

discord stability proxy?



In [ ]:
const llama = importer.import('create llm session')
const {request} = require('gaxios')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const {triggerTyping, createMessage} = importer.import('disrcord api')

async function doHallucinate(interaction) {
  const {llmPrompt} = (await llama)
  let a1 = await llmPrompt('Imagine a scene that is much more exotic, weird, bizarre and detailed with this topic:\n' 
    + interaction.data.options[0].value + '\nFit as much detail as possible in to two or three sentences.')
  if(interaction.guild_id) {
    let q2 = 'Would this prompt be considered to be rated PG, answer Yes or No only?\n' + a1
    let a2 = await llmPrompt(q2)
    if(a2.includes('No')) {
      return await updateInteraction({
        content: 'I will not.'
      }, interaction.id, interaction.token)
    }
  }
  let buff = await doStableRequest(a1)
  return await patchInteractionImageAttachment(interaction.data.options[0].value + '* -> *' + a1, buff, interaction.token)
}

async function doStableRequest(prompt) {
  let width = 1024
  if(prompt.includes('View360')) {
    width = 2048
  }
  let result = await request({
    url: 'http://127.0.0.1:7860/sdapi/v1/txt2img',
    method: 'POST',
    headers: {
      'Content-Type': 'application/json'
    },
    data: JSON.stringify({
      prompt: prompt,
      negative_prompt: 'bad hands, bad feet, bad faces, bad eyes, bad anatomy, extra limbs, missing limbs, tattoo, statue, picture frame, anime, cartoon, signature, abstract',
      save_images: true,
      "width": width,
      "height": 1024,
      "steps": 30,
      tiling: false,
    })
  })
  let buff = Buffer.from(result.data.images[0], 'base64');
  return buff
}


async function doImagineProxy(interaction) {
  if(interaction.guild_id) {
    const {llmPrompt} = (await llama)
    let q2 = 'Would this prompt be considered to be rated PG, answer Yes or No only?\n' + interaction.data.options[0].value
    let a2 = await llmPrompt(q2)
    if(a2.includes('No')) {
      return await updateInteraction({
        content: 'I will not.'
      }, interaction.id, interaction.token)
    }
  }
  let buff = await doStableRequest(interaction.data.options[0].value)
  //const fs = require('fs')
  //fs.writeFileSync('/Users/briancullinan/Downloads/test.png', buff)
  //console.log(atob(result.data.images[0]))
  return await patchInteractionImageAttachment(interaction.data.options[0].value, buff, interaction.token)
}

async function doInteraction(interaction) {
  const {llmPrompt} = (await llama)
  if(interaction.data.name == 'hello') {
    let a1 = await llmPrompt('hello')
    await updateInteraction({
      content: a1.substring(0, 1800)
    }, interaction.id, interaction.token)
  }
  else if (interaction.data.name == 'prompt') {
    let a1 = await llmPrompt(interaction.data.options[0].value + '\nmake it short.')
    await updateInteraction({
      content: a1.substring(0, 1800)
    }, interaction.id, interaction.token)
  }
}

async function doMention(interaction) {
  const {llmPrompt} = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmPrompt(interaction.content.replace('@' + DEFAULT_APPLICATION) + '\nmake it short.')
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}

async function doPrivate(interaction) {
  const {llmPrompt} = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmPrompt(interaction.content)
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}

module.exports = {
  doInteraction,
  doHallucinate,
  doImagineProxy,
  doMention,
  doPrivate
}



discord pdf converter?


In [ ]:
const os = require('os')
const fs = require('fs')
const {updateInteraction} = importer.import('discord api')
const {savePdf} = importer.import('collect all bookmarks')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const {doBrowserRequest} = importer.import('browser crawler tools')
const {
  safeurl,
  existingCache,
  storeCache,
  readCache,
} = importer.import('domain cache tools')

const TEMP_DIR = os.tmpdir();
let counter = 0

async function doDiscordPdf(interaction) {
  // overwrite old pdfs, just for API, should transfer in memory directly, but oh well
  const cache = existingCache(interaction.data.options[0].value, false)
  const filename = path.join(TEMP_DIR, 'temporary_discord_pdf_' + (counter % 4) + '.pdf')
  if(fs.existsSync(filename))
    fs.unlinkSync(filename)
  counter++
  await doBrowserRequest(interaction.data.options[0].value, 
    readCache.bind(null, cache),
    storeCache.bind(null, cache),
    savePdf.bind(null, filename))
  if(fs.existsSync(filename)) {
    return await patchInteractionImageAttachment(interaction.data.options[0].value, fs.readFileSync(filename), interaction.token, 'application/pdf')
  } else {
    return await updateInteraction('Page saving failed: ' + interaction.data.options[0].value, interaction.id, interaction.token)
  }
}

module.exports = {
  doDiscordPdf
}



discord notebook connector?


In [ ]:
const systemBuffer = importer.import('system usage png buffer')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
const llama = importer.import('llm distill')
const llama2 = importer.import('llm instruct')
const llama3 = importer.import('llm deceive')
const llama4 = importer.import('llm reasoning')
const llama5 = importer.import('llm mesh')
const llama6 = importer.import('llm voice')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')

async function doDistill(interaction) {
  const {llmDistill} = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmDistill(interaction.data.options[0].value)
  const mdHtml = md.render(a1);
  return await patchInteractionImageAttachment(interaction.data.options[0].value, Buffer.from(mdHtml), interaction.token, 'text/html')
}

async function doDeceive(interaction) {
  const {llmDeceive} = (await llama3)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmDeceive('Give the most wrong or incorrect response to, be as deceptive as possible:\n' + interaction.data.options[0].value)
  return await updateInteraction(a1.substring(0, 1800), interaction.id, interaction.token)
}

async function doReason(interaction) {
  const {llmReason} = (await llama4)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmReason(interaction.data.options[0].value)
  return await updateInteraction(a1.substring(0, 1800), interaction.id, interaction.token)
}

async function doMesh(interaction) {
  const {llmMesh} = (await llama5)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmMesh('Create a model in OBJ format for: ' + interaction.data.options[0].value)
  let mesh = a1.substring(a1.indexOf('```obj')).replace(/```obj|```/gi, '')
  return await patchInteractionImageAttachment(interaction.data.options[0].value, Buffer.from(mesh), interaction.token, 'model/obj')
}

async function doVoice(interaction) {
  const {llmVoice} = (await llama6)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmVoice('Convert the text to speech: ' + interaction.data.options[0].value)
  return await patchInteractionImageAttachment(interaction.data.options[0].value, Buffer.from(a1), interaction.token, 'audio/wav')
}

async function doInstruct(interaction) {
  const {llmInstruct} = (await llama2)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmInstruct(interaction.data.options[0].value)
  const mdHtml = md.render(a1);
  return await patchInteractionImageAttachment(interaction.data.options[0].value, Buffer.from(mdHtml), interaction.token, 'text/html')
}

async function doSystemUsage(interaction) {
  let buffer = await systemBuffer()
  await patchInteractionImageAttachment('system usage', buffer, interaction.token)
  let update
  let count = 24
  update = setInterval(async () => {
    count--
    buffer = await systemBuffer()
    await patchInteractionImageAttachment('system usage', buffer, interaction.token)
    if(count == 0) {
      clearInterval(update)
    }
  }, 4000)
}

async function doInterpret(interaction) {
  let code = await importer.interpret(interaction.data.options[0].value)
  await updateInteraction({
    content: code.language + ' code:\n```' + (code.language == 'node' ? 'javascript' : code.language) + '\n' + code.code.substring(0, 1800) + '\n```\n'
  }, interaction.id, interaction.token)
}

async function doArgument(interaction) {
  const argueLlama = importer.import('argue with multiple llms')
  let count = 0
  return Promise.resolve(argueLlama(interaction.data.options[0].value, async (prompt, answer) => {
    if(count == 0) {
      // don't repost original prompt
      await updateInteraction(answer.substring(0, 1800), interaction.id, interaction.token)
    } else {
      //await createMessage(prompt.substring(0, 1800), interaction.channel_id)
      await createMessage(answer.substring(0, 1800), interaction.channel_id)
    }
    count++
  }))
}

module.exports = {
  doInterpret,
  doSystemUsage,
  doDistill,
  doInstruct,
  doDeceive,
  doReason,
  doArgument,
  doMesh,
  doVoice,
}



disord express server?


In [ ]:
//const {doBrowserRequest} = importer.import('browser crawler tools')
//const {
//  safeurl, existingCache, storeCache, readCache,
//} = importer.import('domain cache tools')
const {request} = require('gaxios')
const url = require('url');
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const {request: requestDiscord} = importer.import('discord authorize')
const qs = require('qs');


async function getInstance(instanceId) {
  return await requestDiscord({
    method: 'GET',
    url: `applications/${DEFAULT_APPLICATION}/activity-instances/${instanceId}`
  })
}

async function getToken(req, res, next) {
  //console.log(req.body)
  let result = await request({
    url: DEFAULT_API + 'oauth2/token',
    method: 'POST',
    headers: {
      'Content-Type': 'application/x-www-form-urlencoded',
      //'Authorization': 'Bot ' + TOKEN
    },
    data: qs.stringify({
      client_id: DEFAULT_APPLICATION,
      client_secret: SECRET,
      grant_type: 'authorization_code',
			code: req.body.code,
		})
  })
  //console.log(result)
  res.setHeader('Content-Type', 'application/json')
  return res.send(JSON.stringify({access_token: result.data.access_token}))
}


let instances = {}
let express
try {
  express = require('express')
} catch (e) {
  console.error(e)
}

async function handleRoute(req, res, next) {
  //const cache = existingCache('https://www.briancullinan.com' + req.originalUrl, false)
  try {
    var url_parts = url.parse(req.url, true);
    var query = url_parts.query;
    //console.log(query)
    if(typeof query['frame_id'] != 'undefined' && typeof query['user_id'] != 'undefined') {
      //let instance = await getInstance(query['instance_id'])
      //console.log(instance)
      instances[query['frame_id']] = query['user_id']
    }

    // TODO: lookup user id from cookie?

    let result = await request({
      url: 'https://www.briancullinan.com' + req.originalUrl,
      method: req.method,
      data: req.data,
    })
    //console.log(result)
    let data = result.data
    if(result.headers['content-type'].includes('text/html')) {
      data = data.replace('</head>', `
<script type="text/javascript">
${importer.interpret('discord client code').code.replace('{TOKEN}', TOKEN).replace(/\{CLIENT_ID\}/gi, DEFAULT_APPLICATION)}
</script>
</head>
        `)
    }
    res.setHeader('Content-Type', result.headers['content-type'])
    res.send(data)
  } catch(e) {
    console.error(e)
  }
  /*
  try {
    if(cache[])
    let response = await doBrowserRequest('https://www.briancullinan.com' + req.originalUrl, 
      readCache.bind(null, cache),
      storeCache.bind(null, cache),
      async (response, page) => {
        //if()
        //let html = await page.evaluate(() => document.querySelector('*').innerHTML)
        //console.log(response.headers())
      })
    res.setHeader('Content-Type', response.headers()['content-type'])
    return res.send(await response.buffer())
  } catch (e) {
    console.error(e)
  }
  */
  return next()
  //if(!req.get('Referer')) {
  //} else {
  //  console.log(req.get('Referer'))
  //}
}

let app
if(express) {
  app = express();
  app.use(express.json());
  app.post('/api/token', getToken);
  app.get('*', handleRoute);
}

let server
//let activities = {}

async function discordExpress(activity) {
  if(!express) {
    return false
  }
  if(server) {
    server.close()
  }
  //if(typeof activities[activity.instance_id] == 'undefined') {
  //  activities[activity.instance_id] = []
  //}
  //activities[activity.instance_id] = participants

  return (server = await app.listen(3000, () => {
    console.log('Listening on: ', 3000)
  }))
}

function closeExpress() {
  if(server) {
    server.close()
  }
}

module.exports = {
  discordExpress,
  closeExpress,
  getInstance,
}




discord client code?


In [ ]:

let code
let userId

function sourceWindow() {
  return (window.parent.opener || window.parent)
}

document.addEventListener('click', (evt) => {
  if(evt.target.tagName == 'A' && evt.target.href.includes('://')) {
    sourceWindow().postMessage([1, {
      cmd: 'OPEN_EXTERNAL_LINK', 
      args: {url: evt.target.href}, 
      nonce: Math.random() * 1000,
      transfer: void 0
    }], document.referrer || '*')
  }
  return true
})

let result = {}
let params = (window.location.search.split('?')[1] || '').split('&')
for(var param in params) {
  if (params.hasOwnProperty(param)) {
    let paramParts = params[param].split('=')
    result[paramParts[0]] = decodeURIComponent(paramParts[1] || "")
  }
}

window.addEventListener('message', async (evt) => {
  debugger

  if(evt.data[1].cmd == 'AUTHORIZE') {
    code = evt.data[1].data.code

    debugger

    const response = await fetch("/.proxy/api/token", {
      method: "POST",
      headers: {
        "Content-Type": "application/json",
      },
      body: JSON.stringify({
        code,
      }),
    });
    const { access_token } = await response.json();
  
    sourceWindow().postMessage([1, {
      cmd: 'AUTHENTICATE',
      args: {
        access_token: access_token
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], document.referrer || '*')
  }

  if(evt.data[1].cmd == 'AUTHENTICATE') {
    userId = evt.data[1].data.user.id

    if(typeof result.frame_id != 'undefined' && typeof result.user_id == 'undefined')
      window.location = window.location + '&user_id=' + userId
  }
});

document.addEventListener("DOMContentLoaded", (event) => {

  if(typeof result.frame_id == 'undefined') {
    return
  }


  setTimeout(() => {
      
    sourceWindow().postMessage([0, {
      v: 1,
      encoding: 'json',
      client_id: '{CLIENT_ID}',
      frame_id: result.frame_id,
    }], document.referrer || '*')


  }, 1000)

  setTimeout(() => {
      
    sourceWindow().postMessage([1, {
      cmd: 'AUTHORIZE',
      args: {
        client_id: '{CLIENT_ID}',
        prompt: 'none',
        response_type: 'code',
        scope: ['applications.commands', 'identify', 'guilds.members.read', 'rpc.activities.write'],
        state: '',
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], document.referrer || '*')

  }, 2000)

})
